---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [3]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [4]:
def answer_one():
        
    movies = nx.read_edgelist('Employee_Movie_Choices.txt',delimiter='\t')
    
    return movies # Your Answer Here

#g = answer_one()
#plot_graph(g)
#!cat Employee_Movie_Choices.txt

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [5]:
def answer_two():
    
    g = answer_one()
    
    for node in g.nodes():
        if node in employees:
            g.add_node(node,type='employee')
        if node in movies:
            g.add_node(node,type='movie')
    
    #print(g.nodes(data=True))
    
    return g # Your Answer Here

#g2=answer_two()

#for node in g2.nodes(data=True):
#    print(node)

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [13]:
def answer_three():
        
    a2 = answer_two()
    
    P = bipartite.weighted_projected_graph(a2,employees)
    
    return P # Your Answer Here

#a3=answer_three()
#plot_graph(a3)

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [36]:
def a4_helper(h,a3):

    e1 = h['e1']
    e2 = h['e2']
    
    try:
        r = a3[e1][e2]['weight']
    except:
        r = 0
        
    return r

def answer_four():
    
    # load new employee relationship data
    s_df = pd.read_csv('Employee_Relationships.txt', delim_whitespace=True, 
                   header=None, names=['e1', 'e2', 'score'])
    
    # get weighted projected graph from last question
    a3 = answer_three()
    
    # extract dataframe from graph
    #m_df = pd.DataFrame(a3.edges(data=True), columns=['e1', 'e2', 'weight'])
    # it would be nice if they could enhance edges() to avoid this bit of nonsense
    #m_df['weight'] = m_df['weight'].map(lambda x: x['weight'])
    
    # I think the best way to do this is to use a map
    s_df['weight'] = s_df[['e1','e2']].apply(lambda x: a4_helper(x,a3),axis=1)
    
    #print(s_df)
    
    return s_df[['score','weight']].corr().iloc[0]['weight'] # Your Answer Here

a4 = answer_four()
print(a4)

0.788396222173
